In [3]:
import mysql.connector
import pandas as pd

In [4]:
df=pd.read_csv('Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
print(df.shape)       # Number of rows and columns
print(df.columns)     # List of columns
print(df.isnull().sum())  # Number of nulls per colunm

(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
# Fill 'Age' with median value
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [7]:

df=df[df['Age']>18]
df['Name']=df['Name'].str.upper()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"BRAUND, MR. OWEN HARRIS",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"CUMINGS, MRS. JOHN BRADLEY (FLORENCE BRIGGS TH...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"HEIKKINEN, MISS. LAINA",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"FUTRELLE, MRS. JACQUES HEATH (LILY MAY PEEL)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"ALLEN, MR. WILLIAM HENRY",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
from sqlalchemy import create_engine, text

# Your DB credentials
db_user = "root"
db_password = "maherin"
db_name = "etl_demo"
db_host = "localhost"

# Create SQLAlchemy engine using PyMySQL
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

# Your table creation query wrapped in `text()`
create_table_query = text("""
CREATE TABLE IF NOT EXISTS titanic (
    PassengerId INT,
    Survived INT,
    Pclass INT,
    Name VARCHAR(255),
    Sex VARCHAR(10),
    Age FLOAT,
    SibSp INT,
    Parch INT,
    Ticket VARCHAR(50),
    Fare FLOAT,
    Cabin VARCHAR(50),
    Embarked VARCHAR(5)
)
""")

# Execute table creation
with engine.begin() as connection:
    connection.execute(create_table_query)

# Clean NaN → None
df = df.where(pd.notnull(df), None)

# Insert data
df.to_sql('titanic', con=engine, if_exists='append', index=False)
print("Data inserted successfully!")


Data inserted successfully!
